In [5]:
#如果是Keras 的 话， 直接在代码最前面加入这几行代码即可
from keras import backend as K
# set GPU memory 
if('tensorflow' == K.backend()):
    import tensorflow as tf
    from keras.backend.tensorflow_backend import set_session

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

Using TensorFlow backend.


In [1]:
def extract_text(x):
    soup = BeautifulSoup(x, 'html.parser')
    return soup.text

def clear_text_punctuation(x):
    x = html.unescape(x)  # 反转义字符,显示真实内容。
    x = HanziConv.toSimplified(x) # 汉字繁体 转成 简体
    x = x.strip() # 去除字符首尾的指定字符
    x = "".join([i for i in x if i not in punctuation]) # 遍历评论每个字符，若是标点，去掉用空字符连接
    return re.sub(r'\s+', ' ', x) # 把x通过正则表达式 (\s 代表空格，+ 代表多个)，替换成空字符   
# 数字和字母过滤方法，及一些特殊符号

In [2]:
import jieba
def jieba_word(x):
    return [i for i in jieba.cut(x)]

In [5]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
punctuation=hanzi.punctuation + string.punctuation
punctuation = set([i for i in punctuation])
import html
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from tqdm import tqdm

def get_comment_word2vec(sql, vecName):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',db='game_source',charset='utf8mb4')
    df_comment = pd.read_sql(sql, conn)  # 返回的是 dataFrame 数据结构
    conn.close()
    
    df_comment['clear_content'] = df_comment['content'].apply(clear_text_punctuation)
    df_comment['jieba_word'] = df_comment['clear_content'].apply(jieba_word)

    words = tqdm([x for x in df_comment['jieba_word']])
    model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)
    # 取出 wv 属性，是 Word2VecKeyedVectors 
    # 别直接存储 model, 虽然可以追加，但是追加训练会不包括新词，没处理好该问题
    # wv 存储，耗内存少,具体区别 有待进一步体会
    word_vectors = model.wv
    word_vectors.save(vecName)

In [6]:
# 分别训练 qq_mobile taptap渠道的 word2vec
get_comment_word2vec('select content from s_game_comments_qq_game','qq_comments_vector')
get_comment_word2vec('select content from s_game_comments_taptap_game','taptap_comments_vector')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.605 seconds.
Prefix dict has been built succesfully.
100%|██████████| 3324317/3324317 [00:26<00:00, 127748.82it/s]


In [14]:
# 查看 应用宝的评论文本的 模型
from gensim.models import KeyedVectors
qq_vector = KeyedVectors.load("taptap_comments_vector", mmap='r')

In [15]:
qq_vector.most_similar(positive=['男人', '国王'], negative=['女人'])

[('领主', 0.7517091035842896),
 ('将军', 0.681869626045227),
 ('魔王', 0.6727608442306519),
 ('勇士', 0.655035674571991),
 ('勇者', 0.6496292352676392),
 ('信徒', 0.6355671286582947),
 ('农夫', 0.6272660493850708),
 ('矿工', 0.6268424391746521),
 ('族人', 0.6232225894927979),
 ('恶魔', 0.6221526861190796)]

In [16]:
qq_vector.similarity('非常', '很')

0.85668097597747572

In [17]:
qq_vector.similar_by_word("求生")

[('监察', 0.7858465909957886),
 ('逃生', 0.7732838988304138),
 ('大逃杀', 0.6838356852531433),
 ('逢生', 0.6438950300216675),
 ('舔食', 0.6280076503753662),
 ('潜兵', 0.6231508255004883),
 ('追猎', 0.6077479124069214),
 ('腐殖', 0.6076853275299072),
 ('逃杀', 0.6061335802078247),
 ('岛重', 0.600558876991272)]

In [18]:
qq_vector.n_similarity(['我', '很','喜欢','这个','游戏'], ['这个', '游戏','很','不错','挺','赞'])

0.78907560602404736

In [19]:
qq_vector.distance("很", "特别")

0.19388726693426606

In [20]:
qq_vector['求生']

memmap([-1.33626497, -4.60259914,  1.01764572, -2.87018847,  1.13950217,
       -0.6130681 , -1.50238705,  3.76427627,  0.48720589,  1.36765242,
       -1.80725658,  1.06561112,  0.27709526,  0.29756361,  1.0447638 ,
       -3.02857852,  0.82067448, -4.27022028, -0.51740491, -1.67667592,
       -0.44198424, -2.25331998,  1.43272984, -3.63497663,  4.70034695,
       -0.44074306, -0.21645303,  0.12574375,  2.73682618, -1.50603795,
       -2.01849079,  3.47720718, -0.67992592,  1.2328881 ,  1.91297317,
        3.23571277,  3.15171337,  3.19843817, -3.72199416, -1.44380736,
        0.17890073,  1.15622211,  1.15375125, -0.44042143,  0.91262478,
       -0.65663034,  4.39379072,  1.57259202,  3.86545992,  0.2163209 ,
        4.85433578, -0.2512745 , -0.44280222, -0.44292584, -0.68006843,
        0.16226456,  1.41585958,  2.25660038,  0.65347105, -2.99246645,
       -4.10421562, -1.71246135,  2.90390587, -4.99090672, -2.73074841,
        3.15502572,  0.43288565, -4.90127754,  0.28281939, -0.9

In [2]:
###################################
#如果是Keras 的 话， 直接在代码最前面加入这几行代码即可
from keras import backend as K
# set GPU memory 
if('tensorflow' == K.backend()):
    import tensorflow as tf
    from keras.backend.tensorflow_backend import set_session

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

Using TensorFlow backend.


In [3]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
punctuation=hanzi.punctuation + string.punctuation 
import html
conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',db='game_source',charset='utf8mb4')
sql = "SELECT source,game_id, game_name, description, category, tags FROM s_game_detail where source='taptap'"
df_detail = pd.read_sql(sql, conn)  # 返回的是 dataFrame 数据结构
conn.close()
df_detail.head()

,source,game_id,game_name,description,category,tags
0,taptap,1,钢琴块2 （别踩白块儿2 ）,"<div class=""body-description-paragraph"" id=""de...",休闲,"手速,反应力,音乐,有毒,单机,速度,钢琴,魔性,节奏,音游,街机,竖屏,休闲"
1,taptap,2,部落冲突 (Clash of Clans),"<div class=""body-description-paragraph"" id=""de...",策略,"免费,策略,塔防,动作,COC,联机,中文,全球同服,经营,联网"
2,taptap,3,花花传奇,"<div class=""body-description-paragraph"" id=""de...",休闲,"益智,消除,三消,休闲"
3,taptap,4,地铁跑酷,"<div class=""body-description-paragraph"" id=""de...",冒险,"跑酷,单机,免费,动作,冒险,益智,中文,街机,有内购,联网"
4,taptap,5,神庙逃亡2,"<div class=""body-description-paragraph"" id=""de...",动作,"冒险,动作,街机,跑酷,单机,竖屏"


In [9]:
def is_chinese(uchar):
        """判断一个unicode是否是汉字"""
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False
# 判断描述是否 中文字符占一半
def get_chinese_sentence(x):
    l = len(x)+1
    x = "".join([i for i in x if is_chinese(i)])
    return True if len(x)/l >0.8 else False      

In [5]:
# 1. 提取div id  description  text  2.去除 text里面的 html标签 
punctuation = punctuation+ '●™★1234567890 '
punctuation_new = set([i for i in punctuation])
def get_text(description):
    soup = BeautifulSoup(description, 'html.parser')
    try :
        x = soup.find_all("div", attrs={"id": "description"})[0].get_text()
    except:
        return ''
    x = html.unescape(x)  # 反转义字符,显示真实内容。
    x = x.replace('\n', '')
    x = HanziConv.toSimplified(x)  # 汉字繁体 转成 简体
    x = "".join([i for i in x if i not in punctuation_new]) # 遍历评论每个字符，若是标点，去掉用空字符连接
    x = re.sub(r'[A-Za-z]+', '', x) 
    return x if get_chinese_sentence(x) else ' '

In [6]:
import jieba
def jieba_word(x):
    return [i for i in jieba.cut(x)]

In [10]:
df_detail = df_detail[df_detail['description']!=' ']
df_detail['clear_desc'] = df_detail['description'].apply(get_text)
df_detail = df_detail[df_detail['clear_desc']!=' ']
df_detail.head()
df_detail['jieba_word'] = df_detail['clear_desc'].apply(jieba_word)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.602 seconds.
Prefix dict has been built succesfully.


In [11]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from tqdm import tqdm
def get_word(df_comment, vecName):
    words = tqdm([x for x in df_comment['jieba_word']])
    model = Word2Vec(words, size=100, window=5, min_count=1, workers=4)
    # 取出 wv 属性，是 Word2VecKeyedVectors 
    # 别直接存储 model, 虽然可以追加，但是追加训练会不包括新词，没处理好该问题
    # wv 存储，耗内存少,具体区别 有待进一步体会
    word_vectors = model.wv
    word_vectors.save(vecName)

In [12]:
get_word(df_detail, 'taptap_description_vector')

100%|██████████| 26518/26518 [00:00<00:00, 30931.93it/s]
